In [1]:
import os

import pandas as pd

In [2]:
pd.options.display.max_columns = None

In [209]:
kbr_data_path = '../../data/raw/KBR/'
kbr_dir_path = '1_Ankiety_w_gospodarstwach_domowych/'
kbr_file_path = 'Etap_V_1_1_Ankiety_w_gospodarstwach_domowych_Wroclaw.xlsx'
excel_sheet = 'Wrocław_ankiety+podróże'

data = pd.read_excel(os.path.join(kbr_data_path, kbr_dir_path, kbr_file_path), excel_sheet, header=[0,1], engine='openpyxl')

data.columns = [f'{i}_{j}' for i, j in data.columns]
data = data.iloc[0:14241]

if data.shape[1] == len(set(list(data.columns))):
    print('All column names are unique.')
else:
    print('All column names are NOT unique.')

All column names are unique.


In [4]:
data.head()

,Lp_Lp,IDENTYFIKACJA ANKIETY_ID_LOS (numer kwestionariusza),IDENTYFIKACJA ANKIETY_Nr ankietera,IDENTYFIKACJA ANKIETY_Data wypełnienia kwestionariusza,IDENTYFIKACJA ANKIETY_Godzina wypełnienia kwestionariusza,IDENTYFIKACJA ANKIETY_Nr rejonu komunikacyjnego,PREFERENCJE DOT. MOBILNOŚCI MIEJSKIEJ_Wprowadzenie opłat za wjazd do centrum dla wszystkich pojazdów,"PREFERENCJE DOT. MOBILNOŚCI MIEJSKIEJ_Dopuszczenie dojazdu do centrum dla pojazdów, które nie zanieczyszczają środowiska (np. pojazdy elektryczne, hybrydowe lub spełniające najwyższe normy spalania)","PREFERENCJE DOT. MOBILNOŚCI MIEJSKIEJ_Jaka jest Pana/Pani opinia na temat rozwiązań wspierających kursowanie tramwajów i autobusów, takich jak priorytet przejazdu na skrzyżowaniach oraz wydzielone pasy na jezdniach?",PREFERENCJE DOT. MOBILNOŚCI MIEJSKIEJ_Z jakiego środka transportu zbiorowego korzysta Pan/Pani najczęściej?,PREFERENCJE DOT. MOBILNOŚCI MIEJSKIEJ_Dlaczego? \n[tramwaj/autobus],PREFERENCJE DOT. MOBILNOŚCI MIEJSKIEJ_Dlaczego? [Inne]\n[tramwaj/autobus],"PREFERENCJE DOT. MOBILNOŚCI MIEJSKIEJ_Jaka jest Pana/Pani opinia na temat rozwoju systemu kolei aglomeracyjnej na terenie Wrocławia i wokół niego (rozbudowa układu torowego, zwiększenie częstotliwości itd.)?",PREFERENCJE DOT. MOBILNOŚCI MIEJSKIEJ_Jak Pan/Pani ocenia wygodę jazdy pojazdami komunikacji zbiorowej?,PREFERENCJE DOT. MOBILNOŚCI MIEJSKIEJ_Jak Pan/Pani ocenia czystość taboru komunikacji zbiorowej we Wrocławiu?,PREFERENCJE DOT. MOBILNOŚCI MIEJSKIEJ_Jak Pan/Pani ocenia punktualność komunikacji zbiorowej we Wrocławiu?,PREFERENCJE DOT. MOBILNOŚCI MIEJSKIEJ_Jak ocenia Pan/Pani efekty dotychczasowych działań związanych z rozbudową systemu rowerowego we Wrocławiu (dróg i parkingów rowerowych)?,PREFERENCJE DOT. MOBILNOŚCI MIEJSKIEJ_Jakie elementy stanowią dla Pana/Pani największa uciążliwość w codziennych podróżach pieszych po Wrocławiu (można wskazać maksymalnie 3 odpowiedzi) [niekorzystne ustawienia sygnalizacji świetlnej],PREFERENCJE DOT. MOBILNOŚCI MIEJSKIEJ_Jakie elementy stanowią dla Pana/Pani największa uciążliwość w codziennych podróżach pieszych po Wrocławiu (można wskazać maksymalnie 3 odpowiedzi) [brak chodników i konieczność poruszania się jezdnią/poboczem/wydeptaną ścieżką],PREFERENCJE DOT. MOBILNOŚCI MIEJSKIEJ_Jakie elementy stanowią dla Pana/Pani największa uciążliwość w codziennych podróżach pieszych po Wrocławiu (można wskazać maksymalnie 3 odpowiedzi) [zły stan nawierzchni chodników],PREFERENCJE DOT. MOBILNOŚCI MIEJSKIEJ_Jakie elementy stanowią dla Pana/Pani największa uciążliwość w codziennych podróżach pieszych po Wrocławiu (można wskazać maksymalnie 3 odpowiedzi) [zastawianie chodników przez parkujące samochody],PREFERENCJE DOT. MOBILNOŚCI MIEJSKIEJ_Jakie elementy stanowią dla Pana/Pani największa uciążliwość w codziennych podróżach pieszych po Wrocławiu (można wskazać maksymalnie 3 odpowiedzi) [niebezpieczne zachowania kierowców],PREFERENCJE DOT. MOBILNOŚCI MIEJSKIEJ_Jakie elementy stanowią dla Pana/Pani największa uciążliwość w codziennych podróżach pieszych po Wrocławiu (można wskazać maksymalnie 3 odpowiedzi) [zagrożenie ze strony rowerzystów poruszających się chodnikami],PREFERENCJE DOT. MOBILNOŚCI MIEJSKIEJ_Jakie elementy stanowią dla Pana/Pani największa uciążliwość w codziennych podróżach pieszych po Wrocławiu (można wskazać maksymalnie 3 odpowiedzi) [zbyt wysokie krawężniki],PREFERENCJE DOT. MOBILNOŚCI MIEJSKIEJ_Jakie elementy stanowią dla Pana/Pani największa uciążliwość w codziennych podróżach pieszych po Wrocławiu (można wskazać maksymalnie 3 odpowiedzi) [brak bieżącego utrzymania czystości/odśnieżania],PREFERENCJE DOT. MOBILNOŚCI MIEJSKIEJ_Jakie elementy stanowią dla Pana/Pani największa uciążliwość w codziennych podróżach pieszych po Wrocławiu (można wskazać maksymalnie 3 odpowiedzi) [niewystarczająca liczba przejść dla pieszych],"PREFERENCJE DOT. MOBILNOŚCI MIEJSKIEJ_Jakie elementy stanowią dla Pana/Pani największa uciążliwość w codziennych podróżach pieszych po Wrocławiu (można wska

In [220]:
chosen_cols = [
    'IDENTYFIKACJA ANKIETY_ID_LOS (numer kwestionariusza)',
    'IDENTYFIKACJA OSOBY_Nr. osoby w gosp. dom.',
    'IDENTYFIKACJA ANKIETY_Data wypełnienia kwestionariusza',
    'DANE O RESPONDENCIE_Przedział wiekowy',
    'DANE O RESPONDENCIE_Płeć',
    'OPIS PODRÓŻY "ŹRÓDŁO"_Godzina rozpoczęcia podróży\n[gg:mm:ss]',
    'OPIS PODRÓŻY "ŹRÓDŁO"_Z jakiego miejsca',
    'OPIS PODRÓŻY "CEL"_Do jakiego miejsca',
    'OPIS PODRÓŻY "CEL"_Do jakiego miejsca [Inne]',
    '23:59:00_motywacje'
]

data_filtered = data[chosen_cols]

data_filtered.columns = [
    'id', 
    'household_person_id',
    'questionnaire_date',
    'age_bracket', 
    'sex',
    'trip_start_time',
    'source', 
    'destination',
    'what_inne',
    'source-destination'
]

data_filtered.fillna(0, inplace=True)
# data_filtered.dropna(inplace=True)
# data_filtered.reset_index(drop=True, inplace=True)

In [221]:
data_filtered.head()

,id,household_person_id,questionnaire_date,age_bracket,sex,trip_start_time,source,destination,what_inne,source-destination
0,LOS_67-67-306,1.0,2018-04-25,61 i więcej (emeryci kobiety),Kobieta,09:00:00,dom,Inne,SKLEP,dom-Inne
1,LOS_67-67-306,1.0,2018-04-25,61 i więcej (emeryci kobiety),Kobieta,09:40:00,Inne,dom,0,Inne-dom
2,LOS_67-67-109,0.0,2018-04-26,20-24 (wiek studencki),Kobieta,0,0,0,0,-
3,LOS_67-67-109,0.0,2018-04-26,20-24 (wiek studencki),Mężczyzna,0,0,0,0,-
4,LOS_67-67-401,0.0,2018-04-26,61 i więcej (emeryci kobiety),Kobieta,0,0,0,0,-


In [222]:
data_filtered['person_id'] = data_filtered['id'] + data_filtered['questionnaire_date'].astype('str') + data_filtered['household_person_id'].astype('str')
data_filtered['trip_start_time'] = data_filtered['trip_start_time'].astype('str')
data_filtered['source'] = data_filtered['source'].str.replace('Inne', 'inne')
data_filtered['destination'] = data_filtered['destination'].str.replace('Inne', 'inne')
data_filtered['what_inne'] = data_filtered['what_inne'].str.replace('Inne', 'inne')
data_filtered['source-destination'] = data_filtered['source-destination'].str.replace('Inne', 'inne')

In [145]:
# bez inne
data_filtered = data_filtered[
    (~data_filtered['source-destination'].str.contains('inne')) &
    (data_filtered['source-destination'] != '-')
]

In [223]:
# z inne
data_filtered = data_filtered[
    data_filtered['source-destination'] != '-'
]

In [224]:
data_filtered.head()

,id,household_person_id,questionnaire_date,age_bracket,sex,trip_start_time,source,destination,what_inne,source-destination,person_id
0,LOS_67-67-306,1.0,2018-04-25,61 i więcej (emeryci kobiety),Kobieta,09:00:00,dom,inne,SKLEP,dom-inne,LOS_67-67-3062018-04-251.0
1,LOS_67-67-306,1.0,2018-04-25,61 i więcej (emeryci kobiety),Kobieta,09:40:00,inne,dom,NaN,inne-dom,LOS_67-67-3062018-04-251.0
5,LOS_67-67-401,2.0,2018-04-26,45-65 (starsi pracownicy mężczyźni),Mężczyzna,09:00:00,dom,inne,SKLEP,dom-inne,LOS_67-67-4012018-04-262.0
6,LOS_67-67-401,2.0,2018-04-26,45-65 (starsi pracownicy mężczyźni),Mężczyzna,09:40:00,inne,dom,NaN,inne-dom,LOS_67-67-4012018-04-262.0
7,LOS_67-67-533,1.0,2018-04-26,25-44 (młodsi pracownicy),Kobieta,18:00:00,dom,inne,KLUB WSPINACZKOWY,dom-inne,LOS_67-67-5332018-04-261.0


In [225]:
data_filtered = data_filtered.sort_values(by=['person_id', 'trip_start_time'])

In [226]:
df1 = (data_filtered.groupby('person_id')['source-destination']
       .apply(lambda x: ' => '.join(set(x.dropna())))
       .reset_index())

In [238]:
df1.shape

(5571, 2)

In [240]:
data_filtered[
    data_filtered['source-destination'] == 'dom-dom'
].shape

(20, 11)

In [237]:
df1['source-destination'].value_counts()[:50]

dom-praca => praca-dom                                                              1907
dom-inne => inne-dom                                                                1906
szkoła-dom => dom-szkoła                                                             589
uczelnia-dom => dom-uczelnia                                                         233
dom-praca => inne-dom => praca-inne                                                  146
dom-inne => dom-praca => inne-dom => praca-dom                                       133
dom-inne => inne-dom => inne-inne                                                    110
dom-inne => szkoła-dom => inne-dom => dom-szkoła                                      44
dom-praca                                                                             44
dom-inne                                                                              42
dom-uczelnia => inne-dom => uczelnia-inne                                             37
inne-dom             

In [234]:
list(df1['source-destination'].value_counts().to_dict().keys())[:20]

['dom-praca => praca-dom',
 'dom-inne => inne-dom',
 'szkoła-dom => dom-szkoła',
 'uczelnia-dom => dom-uczelnia',
 'dom-praca => inne-dom => praca-inne',
 'dom-inne => dom-praca => inne-dom => praca-dom',
 'dom-inne => inne-dom => inne-inne',
 'dom-inne => szkoła-dom => inne-dom => dom-szkoła',
 'dom-praca',
 'dom-inne',
 'dom-uczelnia => inne-dom => uczelnia-inne',
 'inne-dom',
 'szkoła-inne => inne-dom => dom-szkoła',
 'dom-praca => inne-dom => inne-inne => praca-inne',
 'szkoła-praca => praca-dom => dom-szkoła',
 'uczelnia-dom => dom-uczelnia => inne-dom => dom-inne',
 'dom-dom',
 'dom-praca => praca-praca => praca-dom',
 'dom-inne => inne-praca => praca-dom',
 'praca-dom']

In [ ]:
var_map_dict = {
    'dom-praca => praca-dom': 'praca',
    'dom-inne => inne-dom': 'inne',
    'szkoła-dom => dom-szkoła': 'szkola',
    'uczelnia-dom => dom-uczelnia': 'uczelnia',
    'dom-praca => inne-dom => praca-inne': 'inne-praca',
    'dom-inne => dom-praca => inne-dom => praca-dom': 'dom-inne-praca',
    'dom-inne => inne-dom => inne-inne': 'inne-inne',
    'dom-inne => szkoła-dom => inne-dom => dom-szkoła': 'inne-szkola',
    'dom-praca': 'praca',
    'dom-inne': 'inne',
    'dom-uczelnia => inne-dom => uczelnia-inne': 'inne-uczelnia',
    'inne-dom': 'inne',
    'szkoła-inne => inne-dom => dom-szkoła': 'inne-szkola',
    'dom-praca => inne-dom => inne-inne => praca-inne': 'inne-inne-praca',
    'szkoła-praca => praca-dom => dom-szkoła': 'praca-szkola',
    'uczelnia-dom => dom-uczelnia => inne-dom => dom-inne': 'dom-inne-uczelnia',
    'dom-dom': 'dom',
    'dom-praca => praca-praca => praca-dom': 'dom-praca-dom',
    'dom-inne => inne-praca => praca-dom': 'dom-praca-dom',
    'praca-dom': 'dom-praca-dom',
}

df1.replace({'source-destination': var_map_dict}, inplace=True)

In [230]:
df1['source-destination'].value_counts()

dom-praca => praca-dom                                                              1907
dom-inne => inne-dom                                                                1906
szkoła-dom => dom-szkoła                                                             589
uczelnia-dom => dom-uczelnia                                                         233
dom-praca => inne-dom => praca-inne                                                  146
                                                                                    ... 
szkoła-szkoła => praca-szkoła                                                          1
dom-uczelnia => dom-praca => praca-dom                                                 1
uczelnia-dom => dom-uczelnia => uczelnia-uczelnia                                      1
dom-inne => dom-uczelnia => dom-szkoła => inne-dom => uczelnia-dom => szkoła-dom       1
praca-uczelnia => dom-praca                                                            1
Name: source-destinat

In [228]:
data_filtered['destination'].value_counts()

dom         5897
inne        3050
praca       2500
szkoła       798
uczelnia     353
Name: destination, dtype: int64

In [29]:
data_inne['what_inne'].value_counts()[:50]

inne                    180
SKLEP                   133
sklep                    96
CENTRUM HANDLOWE         79
zakupy                   40
Sklep                    37
przystanek               36
PRZYCHODNIA              25
PARK                     20
RYNEK                    18
OGRÓDKI DZIAŁKOWE        17
PRZEDSZKOLE              15
Biedronka                15
ODWIEDZINY               15
lekarz                   14
Plac Grunwaldzki         14
SZPITAL                  12
Rynek                    12
Przychodnia              12
Zakupy spożywcze         11
Galeria Dominikanska     10
Plac Dominikanski        10
osobisty                  9
rodzina                   9
przychodnia               9
przedszkole               8
park                      8
Lidl                      7
szpital                   7
Przedszkole               7
zoo                       7
Pasaz Grunwaldzki         7
znajomi                   7
APTEKA                    6
Kościół                   6
KOŚCIÓŁ             

In [235]:
data_inne['what_inne'].value_counts()[-50:]

FRYZJER                                                 1
To był spacer nad Odrę, pieszo nad rzeke                1
Poczta glowna                                           1
TARGOWISKO                                              1
Śródmieście Rynek                                       1
PODWOŻENIE                                              1
dom corki                                               1
ODPROWADZANIE DZIECKA                                   1
ogr botaniczny                                          1
LEKARZ                                                  1
Pętle                                                   1
Do siostry - opiekuje się nią                           1
Rossman                                                 1
dworzec - podróż pociągiem                              1
SPACER                                                  1
Kościół Rzymskokatolicki                                1
Wały nad Odra                                           1
Zoo           

# Travel destination grouped by age_bracket-sex combination

In [210]:
kbr_data_path = '../../data/raw/KBR/'
kbr_dir_path = '1_Ankiety_w_gospodarstwach_domowych/'
kbr_file_path = 'Etap_V_1_1_Ankiety_w_gospodarstwach_domowych_Wroclaw.xlsx'
excel_sheet = 'Wrocław_ankiety+podróże'

data = pd.read_excel(os.path.join(kbr_data_path, kbr_dir_path, kbr_file_path), excel_sheet, header=[0,1], engine='openpyxl')

data.columns = [f'{i}_{j}' for i, j in data.columns]
data = data.iloc[0:14241]

if data.shape[1] == len(set(list(data.columns))):
    print('All column names are unique.')
else:
    print('All column names are NOT unique.')

All column names are unique.


In [211]:
data.head()

,Lp_Lp,IDENTYFIKACJA ANKIETY_ID_LOS (numer kwestionariusza),IDENTYFIKACJA ANKIETY_Nr ankietera,IDENTYFIKACJA ANKIETY_Data wypełnienia kwestionariusza,IDENTYFIKACJA ANKIETY_Godzina wypełnienia kwestionariusza,IDENTYFIKACJA ANKIETY_Nr rejonu komunikacyjnego,PREFERENCJE DOT. MOBILNOŚCI MIEJSKIEJ_Wprowadzenie opłat za wjazd do centrum dla wszystkich pojazdów,"PREFERENCJE DOT. MOBILNOŚCI MIEJSKIEJ_Dopuszczenie dojazdu do centrum dla pojazdów, które nie zanieczyszczają środowiska (np. pojazdy elektryczne, hybrydowe lub spełniające najwyższe normy spalania)","PREFERENCJE DOT. MOBILNOŚCI MIEJSKIEJ_Jaka jest Pana/Pani opinia na temat rozwiązań wspierających kursowanie tramwajów i autobusów, takich jak priorytet przejazdu na skrzyżowaniach oraz wydzielone pasy na jezdniach?",PREFERENCJE DOT. MOBILNOŚCI MIEJSKIEJ_Z jakiego środka transportu zbiorowego korzysta Pan/Pani najczęściej?,PREFERENCJE DOT. MOBILNOŚCI MIEJSKIEJ_Dlaczego? \n[tramwaj/autobus],PREFERENCJE DOT. MOBILNOŚCI MIEJSKIEJ_Dlaczego? [Inne]\n[tramwaj/autobus],"PREFERENCJE DOT. MOBILNOŚCI MIEJSKIEJ_Jaka jest Pana/Pani opinia na temat rozwoju systemu kolei aglomeracyjnej na terenie Wrocławia i wokół niego (rozbudowa układu torowego, zwiększenie częstotliwości itd.)?",PREFERENCJE DOT. MOBILNOŚCI MIEJSKIEJ_Jak Pan/Pani ocenia wygodę jazdy pojazdami komunikacji zbiorowej?,PREFERENCJE DOT. MOBILNOŚCI MIEJSKIEJ_Jak Pan/Pani ocenia czystość taboru komunikacji zbiorowej we Wrocławiu?,PREFERENCJE DOT. MOBILNOŚCI MIEJSKIEJ_Jak Pan/Pani ocenia punktualność komunikacji zbiorowej we Wrocławiu?,PREFERENCJE DOT. MOBILNOŚCI MIEJSKIEJ_Jak ocenia Pan/Pani efekty dotychczasowych działań związanych z rozbudową systemu rowerowego we Wrocławiu (dróg i parkingów rowerowych)?,PREFERENCJE DOT. MOBILNOŚCI MIEJSKIEJ_Jakie elementy stanowią dla Pana/Pani największa uciążliwość w codziennych podróżach pieszych po Wrocławiu (można wskazać maksymalnie 3 odpowiedzi) [niekorzystne ustawienia sygnalizacji świetlnej],PREFERENCJE DOT. MOBILNOŚCI MIEJSKIEJ_Jakie elementy stanowią dla Pana/Pani największa uciążliwość w codziennych podróżach pieszych po Wrocławiu (można wskazać maksymalnie 3 odpowiedzi) [brak chodników i konieczność poruszania się jezdnią/poboczem/wydeptaną ścieżką],PREFERENCJE DOT. MOBILNOŚCI MIEJSKIEJ_Jakie elementy stanowią dla Pana/Pani największa uciążliwość w codziennych podróżach pieszych po Wrocławiu (można wskazać maksymalnie 3 odpowiedzi) [zły stan nawierzchni chodników],PREFERENCJE DOT. MOBILNOŚCI MIEJSKIEJ_Jakie elementy stanowią dla Pana/Pani największa uciążliwość w codziennych podróżach pieszych po Wrocławiu (można wskazać maksymalnie 3 odpowiedzi) [zastawianie chodników przez parkujące samochody],PREFERENCJE DOT. MOBILNOŚCI MIEJSKIEJ_Jakie elementy stanowią dla Pana/Pani największa uciążliwość w codziennych podróżach pieszych po Wrocławiu (można wskazać maksymalnie 3 odpowiedzi) [niebezpieczne zachowania kierowców],PREFERENCJE DOT. MOBILNOŚCI MIEJSKIEJ_Jakie elementy stanowią dla Pana/Pani największa uciążliwość w codziennych podróżach pieszych po Wrocławiu (można wskazać maksymalnie 3 odpowiedzi) [zagrożenie ze strony rowerzystów poruszających się chodnikami],PREFERENCJE DOT. MOBILNOŚCI MIEJSKIEJ_Jakie elementy stanowią dla Pana/Pani największa uciążliwość w codziennych podróżach pieszych po Wrocławiu (można wskazać maksymalnie 3 odpowiedzi) [zbyt wysokie krawężniki],PREFERENCJE DOT. MOBILNOŚCI MIEJSKIEJ_Jakie elementy stanowią dla Pana/Pani największa uciążliwość w codziennych podróżach pieszych po Wrocławiu (można wskazać maksymalnie 3 odpowiedzi) [brak bieżącego utrzymania czystości/odśnieżania],PREFERENCJE DOT. MOBILNOŚCI MIEJSKIEJ_Jakie elementy stanowią dla Pana/Pani największa uciążliwość w codziennych podróżach pieszych po Wrocławiu (można wskazać maksymalnie 3 odpowiedzi) [niewystarczająca liczba przejść dla pieszych],"PREFERENCJE DOT. MOBILNOŚCI MIEJSKIEJ_Jakie elementy stanowią dla Pana/Pani największa uciążliwość w codziennych podróżach pieszych po Wrocławiu (można wska

In [212]:
chosen_cols = [
    'IDENTYFIKACJA ANKIETY_ID_LOS (numer kwestionariusza)',
    'DANE O RESPONDENCIE_Przedział wiekowy',
    'DANE O RESPONDENCIE_Płeć',
    'OPIS PODRÓŻY "ŹRÓDŁO"_Z jakiego miejsca',
    'OPIS PODRÓŻY "CEL"_Do jakiego miejsca'
]

data_filtered = data[chosen_cols]

data_filtered.columns = [
    'id', 
    'age_bracket', 
    'sex', 
    'source', 
    'destination'
]

data_filtered.fillna(0, inplace=True)
# data_filtered.dropna(inplace=True)
# data_filtered.reset_index(drop=True, inplace=True)

In [213]:
age_bracket_map_dict = {
    '6-15 (dzieci)': '6-15',
    '16-19 (młodzież)': '16-19',
    '20-24 (wiek studencki)': '20-24',
    '25-44 (młodsi pracownicy)': '25-44',
    '45-60 (starsi pracownicy kobiety)': '45-60',  # '45-60(K)',
    '45-65 (starsi pracownicy mężczyźni)': '45-65',  # '45-65(M)',
    '61 i więcej (emeryci kobiety)': '61-x',  # '61-(K)',
    '66 i więcej (emeryci mężczyźni)': '66-x'  # '66-(M)'
}
data_filtered.replace({'age_bracket': age_bracket_map_dict}, inplace=True)

sex_map_dict = {
    'Kobieta': 'K',
    'Mężczyzna': 'M'
}
data_filtered.replace({'sex': sex_map_dict}, inplace=True)

data_filtered['age_sex_comb'] = data_filtered['age_bracket'] + '_' + data_filtered['sex']

In [214]:
data_filtered['source'] = data_filtered['source'].str.replace('Inne', 'inne')
data_filtered['destination'] = data_filtered['destination'].str.replace('Inne', 'inne')

In [215]:
count_series = data_filtered.groupby(['age_sex_comb', 'destination']).size()
destination_df = count_series.to_frame(name = 'count').reset_index()

In [216]:
destination_df = destination_df.pivot(index='age_sex_comb', columns='destination')['count']
destination_df = destination_df.fillna(0)

In [217]:
destination_df

destination,dom,inne,praca,szkoła,uczelnia
age_sex_comb,,,,,
16-19_K,88.0,28.0,7.0,59.0,12.0
16-19_M,129.0,36.0,5.0,93.0,12.0
20-24_K,280.0,143.0,57.0,12.0,140.0
20-24_M,259.0,113.0,55.0,8.0,133.0
25-44_K,1023.0,441.0,643.0,81.0,24.0
25-44_M,941.0,292.0,732.0,21.0,26.0
45-60_K,654.0,359.0,386.0,15.0,0.0
45-65_M,750.0,325.0,516.0,10.0,2.0
6-15_K,321.0,61.0,1.0,281.0,0.0


In [218]:
destination_df['sum'] = destination_df['dom'] + destination_df['inne'] + destination_df['praca'] + destination_df['szkoła'] + destination_df['uczelnia']

destination_df['dom'] = destination_df['dom'] / destination_df['sum']
destination_df['inne'] = destination_df['inne'] / destination_df['sum']
destination_df['praca'] = destination_df['praca'] / destination_df['sum']
destination_df['szkoła'] = destination_df['szkoła'] / destination_df['sum']
destination_df['uczelnia'] = destination_df['uczelnia'] / destination_df['sum']

destination_df.drop(columns=['sum'], inplace=True)

In [219]:
destination_df.style.format('{:.3f}')

destination,dom,inne,praca,szkoła,uczelnia
age_sex_comb,,,,,
16-19_K,0.454,0.144,0.036,0.304,0.062
16-19_M,0.469,0.131,0.018,0.338,0.044
20-24_K,0.443,0.226,0.090,0.019,0.222
20-24_M,0.456,0.199,0.097,0.014,0.234
25-44_K,0.462,0.199,0.291,0.037,0.011
25-44_M,0.468,0.145,0.364,0.010,0.013
45-60_K,0.463,0.254,0.273,0.011,0.000
45-65_M,0.468,0.203,0.322,0.006,0.001
6-15_K,0.483,0.092,0.002,0.423,0.000


# APPLE COVID DATA

In [200]:
apple_data_path = '../../data/raw/other/apple'
apple_file_path = 'applemobilitytrends-2021-01-14.csv'

data_apple = pd.read_csv(os.path.join(apple_data_path, apple_file_path))

In [201]:
data_apple.columns

Index(['geo_type', 'region', 'transportation_type', 'alternative_name',
       'sub-region', 'country', '2020-01-13', '2020-01-14', '2020-01-15',
       '2020-01-16',
       ...
       '2021-01-05', '2021-01-06', '2021-01-07', '2021-01-08', '2021-01-09',
       '2021-01-10', '2021-01-11', '2021-01-12', '2021-01-13', '2021-01-14'],
      dtype='object', length=374)

In [202]:
data_apple = data_apple[
    (data_apple['region'] == 'Poland') |
    (data_apple['country'] == 'Poland')
]

In [207]:
data_apple[
    data_apple['alternative_name'] == 'Województwo Dolnośląskie'
]

,geo_type,region,transportation_type,alternative_name,sub-region,country,2020-01-13,2020-01-14,2020-01-15,2020-01-16,2020-01-17,2020-01-18,2020-01-19,2020-01-20,2020-01-21,2020-01-22,2020-01-23,2020-01-24,2020-01-25,2020-01-26,2020-01-27,2020-01-28,2020-01-29,2020-01-30,2020-01-31,2020-02-01,2020-02-02,2020-02-03,2020-02-04,2020-02-05,2020-02-06,2020-02-07,2020-02-08,2020-02-09,2020-02-10,2020-02-11,2020-02-12,2020-02-13,2020-02-14,2020-02-15,2020-02-16,2020-02-17,2020-02-18,2020-02-19,2020-02-20,2020-02-21,2020-02-22,2020-02-23,2020-02-24,2020-02-25,2020-02-26,2020-02-27,2020-02-28,2020-02-29,2020-03-01,2020-03-02,2020-03-03,2020-03-04,2020-03-05,2020-03-06,2020-03-07,2020-03-08,2020-03-09,2020-03-10,2020-03-11,2020-03-12,2020-03-13,2020-03-14,2020-03-15,2020-03-16,2020-03-17,2020-03-18,2020-03-19,2020-03-20,2020-03-21,2020-03-22,2020-03-23,2020-03-24,2020-03-25,2020-03-26,2020-03-27,2020-03-28,2020-03-29,2020-03-30,2020-03-31,2020-04-01,2020-04-02,2020-04-03,2020-04-04,2020-04-05,2020-04-06,2020-04-07,2020-04-08,2020-04-09,2020-04-10,2020-04-11,2020-04-12,2020-04-13,2020-04-14,2020-04-15,2020-04-16,2020-04-17,2020-04-18,2020-04-19,2020-04-20,2020-04-21,2020-04-22,2020-04-23,2020-04-24,2020-04-25,2020-04-26,2020-04-27,2020-04-28,2020-04-29,2020-04-30,2020-05-01,2020-05-02,2020-05-03,2020-05-04,2020-05-05,2020-05-06,2020-05-07,2020-05-08,2020-05-09,2020-05-10,2020-05-11,2020-05-12,2020-05-13,2020-05-14,2020-05-15,2020-05-16,2020-05-17,2020-05-18,2020-05-19,2020-05-20,2020-05-21,2020-05-22,2020-05-23,2020-05-24,2020-05-25,2020-05-26,2020-05-27,2020-05-28,2020-05-29,2020-05-30,2020-05-31,2020-06-01,2020-06-02,2020-06-03,2020-06-04,2020-06-05,2020-06-06,2020-06-07,2020-06-08,2020-06-09,2020-06-10,2020-06-11,2020-06-12,2020-06-13,2020-06-14,2020-06-15,2020-06-16,2020-06-17,2020-06-18,2020-06-19,2020-06-20,2020-06-21,2020-06-22,2020-06-23,2020-06-24,2020-06-25,2020-06-26,2020-06-27,2020-06-28,2020-06-29,2020-06-30,2020-07-01,2020-07-02,2020-07-03,2020-07-04,2020-07-05,2020-07-06,2020-07-07,2020-07-08,2020-07-09,2020-07-10,2020-07-11,2020-07-12,2020-07-13,2020-07-14,2020-07-15,2020-07-16,2020-07-17,2020-07-18,2020-07-19,2020-07-20,2020-07-21,2020-07-22,2020-07-23,2020-07-24,2020-07-25,2020-07-26,2020-07-27,2020-07-28,2020-07-29,2020-07-30,2020-07-31,2020-08-01,2020-08-02,2020-08-03,2020-08-04,2020-08-05,2020-08-06,2020-08-07,2020-08-08,2020-08-09,2020-08-10,2020-08-11,2020-08-12,2020-08-13,2020-08-14,2020-08-15,2020-08-16,2020-08-17,2020-08-18,2020-08-19,2020-08-20,2020-08-21,2020-08-22,2020-08-23,2020-08-24,2020-08-25,2020-08-26,2020-08-27,2020-08-28,2020-08-29,2020-08-30,2020-08-31,2020-09-01,2020-09-02,2020-09-03,2020-09-04,2020-09-05,2020-09-06,2020-09-07,2020-09-08,2020-09-09,2020-09-10,2020-09-11,2020-09-12,2020-09-13,2020-09-14,2020-09-15,2020-09-16,2020-09-17,2020-09-18,2020-09-19,2020-09-20,2020-09-21,2020-09-22,2020-09-23,2020-09-24,2020-09-25,2020-09-26,2020-09-27,2020-09-28,2020-09-29,2020-09-30,2020-10-01,2020-10-02,2020-10-03,2020-10-04,2020-10-05,2020-10-06,2020-10-07,2020-10-08,2020-10-09,2020-10-10,2020-10-11,2020-10-12,2020-10-13,2020-10-14,2020-10-15,2020-10-16,2020-10-17,2020-10-18,2020-10-19,2020-10-20,2020-10-21,2020-10-22,2020-10-23,2020-10-24,2020-10-25,2020-10-26,2020-10-27,2020-10-28,2020-10-29,2020-10-30,2020-10-31,2020-11-01,2020-11-02,2020-11-03,2020-11-04,2020-11-05,2020-11-06,2020-11-07,2020-11-08,2020-11-09,2020-11-10,2020-11-11,2020-11-12,2020-11-13,2020-11-14,2020-11-15,2020-11-16,2020-11-17,2020-11-18,2020-11-19,2020-11-20,2020-11-21,2020-11-22,2020-11-23,2020-11-24,2020-11-25,2020-11-26,2020-11-27,2020-11-28,2020-11-29,2020-11-30,2020-12-01,2020-12-02,2020-12-03,2020-12-04,2020-12-05,2020-12-06,2020-12-07,2020-12-08,2020-12-09,2020-12-10,2020-12-11,2020-12-12,2020-12-13,2020-12-14,2020-12-15,2020-12-16,2020-12-17,2020-12-18,2020-12-19,2020-12-20,2020-12-21,2020-12-22,2020-12-23,2020-12-24,2020-12-25,2020-12-26,2020-12-27,2020-12-28,2020-12-29,2020-12-30,2020-12-31,2021-01-01,2021-01-02,2021-01-03,